<a href="https://colab.research.google.com/github/wikibook/machine-learning/blob/2.0/jupyter_notebook/5.다층퍼셉트론_MNIST_손글씨_예측모델.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

# 항상 같은 결과를 갖기 위해 랜덤 시드 설정
tf.random.set_seed(678)

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from IPython.display import Image

# 다층퍼셉트론 구조
텐서플로우로 아래의 다층퍼셉트론을 구현해보도록 하겠습니다.

In [21]:
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/dropout.png", width=500, height=250)

# MNIST 데이터 불러오기

In [22]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [23]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [24]:
print("train data has " + str(x_train.shape[0]) + " samples")
print("every train data is " + str(x_train.shape[1]) 
      + " * " + str(x_train.shape[2]) + " image")

train data has 60000 samples
every train data is 28 * 28 image


**0** 부터 **255** 까지의 그레이 스케일을 확인할 수 있습니다.

In [25]:
# sample to show gray scale values
print(x_train[0][8])

[  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
   0   0   0   0   0   0   0   0   0   0]


**0** 부터 **9**까지의 이미지에 해당하는 숫자를 확인할 수 있습니다.

In [26]:
# sample to show labels for first train data to 10th train data
print(y_train[0:9])

[5 0 4 1 9 2 1 3 1]


테스트 데이터는 **10000** 개의 샘플을 가지고 있습니다.  
모든 테스트 데이터는 **28 * 28** 의 이미지입니다.  

In [27]:
print("test data has " + str(x_test.shape[0]) + " samples")
print("every test data is " + str(x_test.shape[1]) 
      + " * " + str(x_test.shape[2]) + " image")

test data has 10000 samples
every test data is 28 * 28 image


# 데이터 정규화
데이터 정규화는 보통 학습 시간을 단축하고, 더 나은 성능을 구하도록 도와줍니다.  
MNIST 데이터의 모든 값은 0부터 255의 범위 안에 있으므로, 255로 값을 나눠줌으로써, 모든 값을 0부터 1 사이의 값으로 정규화합니다.  

In [12]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

# 텐서플로우 다층퍼셉트론 구현하기
텐서플로우로 아래의 다층퍼셉트론을 구현해보도록 하겠습니다.

In [13]:
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/dropout.png", width=500, height=250)

## 2차원 데이터를 1차원 데이터로 변경하기 (Flatten)
다층퍼셉트론의 입력 레이어에 데이터를 넣기 위해서 2d tensor (28, 28)인 데이터를,  
1d tensor (28*28, 1)의  형태로 바꿔줍니다.  
이 말은 행렬 형태의 데이터를 배열 형태의 데이터로 변경한다는 의미와 같습니다.

In [29]:
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/reshape_mnist.png", width=300, height=150)

In [36]:
model = Sequential([
    Flatten(input_shape=(28, 28)), # 데이터 차원 변경
    Dense(256, activation='relu'), # 첫번째 히든 레이어 (h1)
    Dense(128, activation='relu'), # 두번째 히든 레이어 (h2)
    Dropout(0.1), # 두번째 히든 레이어(h2)에 드랍아웃(10%) 적용
    Dense(10), # 세번째 히든 레이어 (logit)
    Activation('softmax') # softmax layer
])

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               200960    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
________________________________________________

첫번째 레이어에 784개의 입력을 받는 256개의 노드가 존재하고, 노드마다 편향값 하나씩 존재하므로,  
784 * 256 + 256 = 200960의 파라미터가 존재합니다.  
flatten과 softmax는 노드가 없으므로 파라미터가 존재하지 않는 것을 확인할 수 있습니다.

In [38]:
"""
sparse_categorical_crossentropy:
레이블을 원 핫 인코딩으로 자동으로 변경하여 크로스 엔트로피 측정합니다.
"""
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 조기 종료 (Early Stopping)

매 주기(Epoch)마다 검증 데이터로 검증 정확도를 측정합니다.  
검증 정확도가 5번 연속으로 개선되지 않을 시, 조기 종료를 수행합니다.

In [40]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=False),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True)]

In [41]:
model.fit(x_train, y_train, epochs=300, batch_size=1000, validation_split = 0.1, callbacks=callbacks)

Train on 54000 samples, validate on 6000 samples
Epoch 1/300
54000/54000 [==============================] - 3s 50us/sample - loss: 9.1650 - accuracy: 0.7719 - val_loss: 1.0220 - val_accuracy: 0.9012
Epoch 2/300
54000/54000 [==============================] - 2s 36us/sample - loss: 1.0137 - accuracy: 0.8652 - val_loss: 0.5348 - val_accuracy: 0.8920
Epoch 3/300
54000/54000 [==============================] - 2s 37us/sample - loss: 0.5493 - accuracy: 0.8835 - val_loss: 0.4011 - val_accuracy: 0.9158
Epoch 4/300
54000/54000 [==============================] - 2s 36us/sample - loss: 0.4023 - accuracy: 0.9051 - val_loss: 0.3420 - val_accuracy: 0.9273
Epoch 5/300
54000/54000 [==============================] - 2s 37us/sample - loss: 0.3081 - accuracy: 0.9229 - val_loss: 0.3154 - val_accuracy: 0.9333
Epoch 6/300
54000/54000 [==============================] - 2s 36us/sample - loss: 0.2491 - accuracy: 0.9337 - val_loss: 0.2814 - val_accuracy: 0.9430
Epoch 7/300
54000/54000 [==========================

# 테스트
검증 정확도가 가장 높은 모델을 대상으로 테스트를 진행합니다.

In [42]:
results = model.evaluate(x_test,  y_test, verbose = 0)

In [43]:
print('test loss, test acc:', results)

test loss, test acc: [0.283340558999399, 0.9599]
